<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train7_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import pandas as pd

In [35]:

train7 = pd.read_csv(r'/content/train_data7.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [36]:

train7['Target'] = train7['Target'].replace(['Others'],'Invalid')


In [37]:
train7['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [38]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train7_Xtfidf = tfidf_vect.fit_transform(train7['Sentence'])


In [40]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [41]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [42]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [43]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [44]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [45]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [46]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train7_Xtfidf,train7['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6703539823008849
                  precision    recall  f1-score   support

        Analysis       0.52      0.38      0.44       130
      Conclusion       0.73      0.45      0.56        49
           Facts       0.73      0.91      0.81       475
         Invalid       0.51      0.45      0.47       128
           Issue       0.57      0.45      0.51        53
Rule/Law/Holding       0.61      0.28      0.38        69

        accuracy                           0.67       904
       macro avg       0.61      0.49      0.53       904
    weighted avg       0.65      0.67      0.65       904



In [47]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [48]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 3]
[0.9970934439199719, 0.9999197136469282, 0.9981141817889347, 0.9971758025291991, 0.9837892370933009, 0.9846725704138259, 0.9986376830825934, 0.9940783456585174, 0.9958685469808355, 0.9977829859865415, 0.9960666706151078, 0.9993337948176741, 0.9998437401616056, 0.9913013329619678, 0.9923273036010546, 0.9958469639759431, 0.9956620363038088, 0.9958478110120136, 0.999435984252756, 0.999716308592064, 0.9809306786264299, 0.9886314192227295, 0.99589760569737, 0.9957569909740704, 0.9913194574290392, 0.9972764522366355, 0.9952980713328176, 0.9989168797948019, 0.9970233256301939, 0.9947779923104059, 0.9968643034821904, 0.9887296979238864, 0.9822582462362485, 0.9908668214699287, 0.9999120423267778, 0.9869864941410456, 0.9902195233969615, 0.9884331667573101, 0.9944698529810789, 0.9938077551219286, 0.9994233493457078, 0.9993452341557537, 0.9972770557060642]
[4, 5, 11, 12

In [49]:
unlabel_1=unlabel_1.loc[[4, 5, 11, 12, 14, 17, 18, 19, 21, 23, 24, 29, 31, 35, 36, 37, 39, 40, 45, 48, 49, 50, 51, 52, 55, 56, 60, 62, 68, 69, 70, 72, 74, 75, 76, 78, 79, 84, 87, 89, 94, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
11,1604,police notified deceased found lying mrs moers...,Facts
12,1102,shown vine street drug storeâs supply narcotic...,Facts
14,2901,testimony armstrong showing prior transactions...,Facts


In [50]:
frame_1 = [train7,unlabel_1]
train7_1 = pd.concat(frame_1)
len(train7_1)

3519

In [51]:
x_train_1 = tfidf_vect.transform(train7_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train7_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6676136363636364
                  precision    recall  f1-score   support

        Analysis       0.49      0.46      0.47       102
      Conclusion       0.69      0.46      0.55        39
           Facts       0.75      0.89      0.81       355
         Invalid       0.53      0.52      0.53       110
           Issue       0.61      0.44      0.51        39
Rule/Law/Holding       0.64      0.24      0.35        59

        accuracy                           0.67       704
       macro avg       0.62      0.50      0.54       704
    weighted avg       0.66      0.67      0.65       704



In [52]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 2, 2, 2, 2, 2, 2, 2]
[0.9981949878626287, 0.9992783933586873, 0.9891349515110044, 0.9979205149703969, 0.9974708172559665, 0.9996907350374252, 0.999983982175005, 0.9864693040309971, 0.990248528177974, 0.9989127226292821, 0.9939188792784515, 0.9934442115458774, 0.9993394559549635, 0.9832913871829949, 0.9900995945997546, 0.9995916973523848, 0.9999733433062541, 0.9995824463648239, 0.999660921548155, 0.9876905462444097, 0.9995771238963392, 0.9864310198713734, 0.9949674687207808, 0.9996476704747225, 0.9994983808230343, 0.9998344338317566, 0.99949265365733, 0.9959629817891129, 0.9906075397111369, 0.9985829006542494, 0.9805201189325368, 0.9958653119269246, 0.9843521795145063, 0.9950111436846362, 0.9966433784108418, 0.9933284228177011, 0.9995324326334489, 0.9975864136252526, 0.9991716998634019, 0.9919999098230724, 0.9929495555688211]
[0, 2, 3, 4, 7, 8, 12, 13, 14, 20, 21, 24, 29, 31, 34, 36, 3

In [53]:
unlabel_2 = unlabel_2.loc[[0, 2, 3, 4, 7, 8, 12, 13, 14, 20, 21, 24, 29, 31, 34, 36, 37, 39, 42, 44, 49, 50, 55, 59, 63, 64, 69, 70, 71, 75, 77, 79, 82, 83, 85, 86, 88, 90, 91, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [54]:
frame_2 = [train7_1,unlabel_2]
train7_2 = pd.concat(frame_2)
len(train7_2)

3560

In [55]:
x_train_2 = tfidf_vect.transform(train7_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train7_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6652806652806653
                  precision    recall  f1-score   support

        Analysis       0.46      0.45      0.45       136
      Conclusion       0.74      0.45      0.56        51
           Facts       0.73      0.90      0.81       489
         Invalid       0.56      0.46      0.50       168
           Issue       0.66      0.37      0.47        57
Rule/Law/Holding       0.66      0.31      0.42        61

        accuracy                           0.67       962
       macro avg       0.63      0.49      0.54       962
    weighted avg       0.65      0.67      0.65       962



In [56]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2]
[0.9989796495523605, 0.9982564690954552, 0.9940543339202237, 0.9926940968925864, 0.9997395627048948, 0.9998344987775402, 0.999469677440967, 0.984633722912681, 0.9997974250253407, 0.9920773318325787, 0.9996209312377442, 0.9993922065092357, 0.9981576016389272, 0.9863075449356072, 0.998423005641018, 0.9987811659366201, 0.9994953045710623, 0.9979812739159887, 0.9905856770497838, 0.9989300152482818, 0.9964663668628004, 0.9838246945309884, 0.9898141569529965, 0.9956258845404329, 0.999368593584903, 0.9972714198994151, 0.9997849648427867, 0.9922694852774787, 0.9915241770737039, 0.999809165708977, 0.9961427432006148, 0.9977551734690648, 0.998624514820022, 0.9896054098897221, 0.9967748576262098, 0.9827676510980066, 0.9873053474663537, 0.9995907394229676, 0.9923629525088059]
[0, 2, 3, 4, 7, 8, 12, 18, 20, 26, 27, 29, 30, 34, 35, 36, 43, 46, 48, 52, 53, 54, 56, 57, 63, 64, 65, 66, 

In [57]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 4, 7, 8, 12, 18, 20, 26, 27, 29, 30, 34, 35, 36, 43, 46, 48, 52, 53, 54, 56, 57, 63, 64, 65, 66, 67, 68, 70, 72, 75, 79, 80, 85, 87, 89, 97],:]
frame_3 = [train7_2,unlabel_3]
train7_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train7_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train7_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6611111111111111
                  precision    recall  f1-score   support

        Analysis       0.44      0.39      0.41       101
      Conclusion       0.72      0.47      0.57        45
           Facts       0.74      0.89      0.81       374
         Invalid       0.53      0.47      0.50       116
           Issue       0.63      0.41      0.50        41
Rule/Law/Holding       0.52      0.28      0.36        43

        accuracy                           0.66       720
       macro avg       0.60      0.48      0.52       720
    weighted avg       0.64      0.66      0.64       720



In [58]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 0, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2]
[0.9882965620128394, 0.9969883351503186, 0.9966797613337625, 0.995703733835054, 0.9972795264383353, 0.9871274322285232, 0.9999243726715984, 0.9996843711279085, 0.984633112743895, 0.999728833315802, 0.9801312497860821, 0.9980533967873292, 0.9984555642501457, 0.9986633368471207, 0.9996865546800836, 0.9981424611769301, 0.998424408414934, 0.9983537376830358, 0.9989154423749073, 0.9964927336468645, 0.9997301428085477, 0.99659493635564, 0.9883715804265675, 0.9937544433229341, 0.9971707311058816, 0.9901947888477706, 0.9989539012296056, 0.9878593386954868, 0.9974671092538913, 0.9974327733218292, 0.9947675738075119, 0.999192054037047, 0.9890107026517924, 0.9995253670668086, 0.9991285433731572, 0.9999836269401862, 0.9857246411095809, 0.997060848501179, 0.997617499705821, 0.9969510026174985, 0.9993085422450882, 0.9944385757818922, 0.9989664594921923]
[0, 2, 4, 6, 7, 8,

In [59]:
unlabel_4=unlabel_4.loc[[0, 2, 4, 6, 7, 8, 10, 15, 17, 18, 21, 22, 27, 28, 29, 31, 32, 34, 37, 38, 40, 42, 46, 49, 50, 55, 56, 57, 58, 60, 66, 67, 69, 70, 71, 75, 76, 82, 85, 87, 93, 97, 99],:]

In [60]:
frame_4 = [train7_3,unlabel_4]
train7_4 = pd.concat(frame_4)
len(train7_4)
x_train_4 = tfidf_vect.transform(train7_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train7_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6808943089430894
                  precision    recall  f1-score   support

        Analysis       0.51      0.43      0.47       139
      Conclusion       0.59      0.42      0.49        53
           Facts       0.77      0.88      0.82       517
         Invalid       0.53      0.50      0.51       147
           Issue       0.65      0.54      0.59        67
Rule/Law/Holding       0.55      0.36      0.44        61

        accuracy                           0.68       984
       macro avg       0.60      0.52      0.55       984
    weighted avg       0.66      0.68      0.67       984



In [61]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0]
[0.9992318117270604, 0.9862614162396469, 0.9975233229309387, 0.9977202967807954, 0.9955876504073079, 0.9977809896347525, 0.9947322476682137, 0.9984679244733649, 0.9994127292027912, 0.9877291141846305, 0.986878280029099, 0.9852934653981692, 0.9997320792041872, 0.9974562143290447, 0.9849196981678506, 0.9879294623605092, 0.9993896081640816, 0.9913607795922119, 0.9857397752888741, 0.9984937747837057, 0.9890291286133649, 0.9989225292319315, 0.9889665908778366, 0.9801811268272641, 0.9915221933440274, 0.9874696625712057, 0.9892491540888039, 0.9891469234100398, 0.994313787927541, 0.9997813261983259, 0.9990318225091024, 0.988170907065336, 0.9987298025777629, 0.9981379801777418, 0.990965381924633, 0.9956049563687106, 0.9950271818412774, 0.990472501316731]
[3, 4, 7, 12, 18, 19, 22, 23, 26, 27, 28, 32, 36, 46, 50, 52, 53, 56, 59, 60, 63, 65, 66, 67, 71, 73, 75, 77, 78, 82, 86, 92, 94,

In [62]:
unlabel_5 = unlabel_5.loc[[3, 4, 7, 12, 18, 19, 22, 23, 26, 27, 28, 32, 36, 46, 50, 52, 53, 56, 59, 60, 63, 65, 66, 67, 71, 73, 75, 77, 78, 82, 86, 92, 94, 95, 97, 99, 103, 110],:]

In [63]:
frame_5 = [train7_4,unlabel_5]
train7_5 = pd.concat(frame_5)
len(train7_5)
x_train_5 = tfidf_vect.transform(train7_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train7_5['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.676056338028169
                  precision    recall  f1-score   support

        Analysis       0.48      0.41      0.44       140
      Conclusion       0.59      0.50      0.54        54
           Facts       0.78      0.88      0.83       528
         Invalid       0.49      0.50      0.49       157
           Issue       0.63      0.47      0.54        47
Rule/Law/Holding       0.56      0.37      0.44        68

        accuracy                           0.68       994
       macro avg       0.59      0.52      0.55       994
    weighted avg       0.66      0.68      0.66       994



In [64]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6660268714011516
